In [5]:
!pip install umap-learn
!pip install lightfm

     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 1.2MB 8.7MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=6296af6daacabee5b5bf09202863d254894a5cb5383abf9b4fd43fdf4e1c4fcc
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51351 sha256=0900e8f88cf27aad5c6ee46ca5d8bd44dc50132b2924a3670396d1c552e19716
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent
     |████████████████████████████████| 317kB 5.6MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705350 sha256=1db586965c3210f8e466262cbccbd1f8c4aed9170317dff59e335ea66d7ab348
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3a

In [6]:
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import files
import time
import datetime
from tqdm import tqdm
import random
import umap.umap_ as umap
from gensim.models import Word2Vec
from sklearn import preprocessing
import spacy
import gzip
import json
import gc
import matplotlib.pyplot as plt 
from scipy.sparse import csr_matrix
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
def prediction(id_asin, id_cust, overall):
    interactions_selected = pd.read_excel("/content/drive/MyDrive/Проект 'рекомендательная система для магазина'/amazon/interactions.xlsx")
    product_df_selected = pd.read_excel("/content/drive/MyDrive/Проект 'рекомендательная система для магазина'/amazon/product_features.xlsx")
    new_row = {'asin': id_asin, 'reviewerID': id_cust, 'overall':overall}
    interactions_selected = interactions_selected.append(new_row, ignore_index=True)
    chosen_reviewer = interactions_selected['reviewerID'].unique()

    index = product_df_selected.loc[product_df_selected['asin'] == id_asin].index
    current_overall_mean = float(product_df_selected.iloc[index, 1])
    current_count = float(product_df_selected.iloc[index, 2])
    product_df_selected.at[index, 'overall_mean'] = round((((current_overall_mean*current_count+overall)/(current_count+1))*2)/2)
    product_df_selected.at[index, 'count'] = int(round(((current_count+1)/5.0))*5)

    user_book_interaction = pd.pivot_table(interactions_selected, index='reviewerID', columns='asin', values='overall')
    user_book_interaction = user_book_interaction.fillna(0)
    user_book_interaction_csr = csr_matrix(user_book_interaction.values)
    user_book_interaction_csr

    user_id = list(user_book_interaction.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1

    books_metadata_selected_transformed = pd.get_dummies(product_df_selected, columns = ['overall_mean', 'count'])
    books_metadata_csr = csr_matrix(books_metadata_selected_transformed.drop('asin', axis=1).values)

    purchases_list = {}
    for i in chosen_reviewer:
        temp = interactions_selected[interactions_selected["reviewerID"] == i]["asin"].tolist()
        purchases_list[i] = temp
    
    asin_unique = len(list(user_book_interaction.columns))
    lens = []
    l = list(purchases_list.values())
    for i in l:
        lens.append(len(i))
    lens = sorted(list(set(lens)))
    lens = list(set(list(map(lambda x: round(x/5)*5, lens))))
    lens_norm = preprocessing.normalize([lens], norm='max')
    lens_norm = lens_norm.reshape(-1)

    dicti = {}
    for i in range(len(lens)):
        dicti[lens[i]]=lens_norm[i]

    size = len(purchases_list[id_cust])
    size = round(size/5)*5
    coeff = dicti[size]

    answer_wv = Model_WV(purchases_list, asin_unique,coeff)
    answer_light = Model_LIGHT(user_book_interaction, id_cust, user_dict, user_book_interaction_csr,books_metadata_csr,coeff)

    answer_light = pd.DataFrame(answer_light, columns=['asin','scores'])
    answer_wv = pd.DataFrame(answer_wv, columns=['asin','scores'])
    answer_wv['scores'] =np.array(answer_light['scores'])+ np.array(answer_wv['scores'])
    answer_wv = answer_wv.sort_values(by=['scores'], ascending=False)
    return answer_wv

def aggregate_vectors(products, model_wv):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model_wv[i])
        except KeyError:
            continue
    return np.mean(product_vec, axis=0)

def similar_products(v, model_wv, asin_unique):
    ms = pd.DataFrame(model_wv.similar_by_vector(v, topn= asin_unique+1), columns=['asin', 'scores'])
    ms = ms.sort_values(by=['asin'])
    asin = list(ms['asin'])
    scores=list(ms['scores'])
    lst = []
    for i,j in zip(asin, scores):
        lst.append([i,j])
    return lst



def Model_WV(purchases_list, asin_unique, coeff):
    model_wv = Word2Vec(window = 10, sg = 1, hs = 0,
                    negative = 10, 
                    alpha=0.03, min_alpha=0.0007, min_count=1)

    model_wv.build_vocab(list(purchases_list.values()), progress_per=200)

    model_wv.train(purchases_list.values(), total_examples = model_wv.corpus_count, 
                epochs=10, report_delay=1)
    model_wv.init_sims(replace=True)

    answer_wv = similar_products(aggregate_vectors(purchases_list[id_cust], model_wv),model_wv,asin_unique)
    answer_trans_wv = [x[1] for x in answer_wv]
    normalized_answer_wv = preprocessing.normalize([answer_trans_wv],norm='max')
    normalized_answer_wv =  normalized_answer_wv.reshape(-1)*coeff
    for i in range(len(answer_wv)):
        answer_wv[i][1] = normalized_answer_wv[i]
    return answer_wv

def Model_LIGHT(user_book_interaction, id_cust, user_dict, user_book_interaction_csr,books_metadata_csr,coeff):
    model_FM = LightFM(loss='warp', random_state=2016, learning_rate=0.90, no_components=150, user_alpha=0.000005)
    model_FM = model_FM.fit(user_book_interaction_csr, epochs=100, num_threads=16, verbose=False)

    answer_light = sample_recommendation_user(model_FM, user_book_interaction, id_cust, user_dict,books_metadata_csr)
    answer_light_trans = [x[1] for x in answer_light]
    normalized_answer_light = preprocessing.normalize([answer_light_trans], norm='max')
    normalized_answer_light =  normalized_answer_light.reshape(-1)*(1-coeff)
    for i in range(len(answer_light)):
        answer_light[i][1] = normalized_answer_light[i]
    return answer_light




def sample_recommendation_user(model, interactions, user_id, user_dict, books_metadata_csr,
                              threshold = 0,nrec_items = 5):
    
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.DataFrame(model.predict(user_x,np.arange(n_items), item_features=books_metadata_csr), columns=['scores'])
    scores['asin'] = interactions.columns
    scores = scores.sort_values(by=['asin'])
    asin = list(scores['asin'])
    scores=list(scores['scores'])
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    for i in known_items:
        if i in asin:
            ind = asin.index(i)
            scores[ind]=0
    lst = []
    for i,j in zip(asin, scores):
        lst.append([i,j])
    return lst

id_asin = '0441865704'
id_cust = "A2CT25FY12PYNF"
overall = 2
answer = prediction(id_asin, id_cust, overall)
print(answer['asin'][:5].tolist())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).


['B0027YYMU6', 'B0028Y4H1O', 'B006MPRFJQ', 'B000E204LY', 'B000J691BY']
